# AnalogVNN Demo/Tutorial

Copyright © 2021-present Vivswan Shah (vivswanshah@pitt.edu)

<br>

[![arXiv](https://img.shields.io/badge/arXiv-2210.10048-orange.svg)](https://arxiv.org/abs/2210.10048)
[![PyPI version](https://badge.fury.io/py/analogvnn.svg)](https://badge.fury.io/py/analogvnn)
[![Documentation Status](https://readthedocs.org/projects/analogvnn/badge/?version=stable)](https://analogvnn.readthedocs.io/en/stable/?badge=stable)
[![Python](https://img.shields.io/badge/python-3.7--3.10-blue)](https://badge.fury.io/py/analogvnn)
[![License: MPL 2.0](https://img.shields.io/badge/License-MPL_2.0-blue.svg)](https://opensource.org/licenses/MPL-2.0)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://analogvnn.readthedocs.io/en/v1.0.0/tutorial.html">
      <center>
        <img src="https://analogvnn.readthedocs.io/en/in_progess/_static/analogvnn-logo-square-black.svg" height="32px" />
      </center>
      View on AnalogVNN
    </a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/Vivswan/AnalogVNN/blob/v1.0.0/docs/_static/AnalogVNN_Demo.ipynb">
      <center>
        <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
      </center>
      Run in Google Colab
    </a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/Vivswan/AnalogVNN/blob/v1.0.0/docs/_static/AnalogVNN_Demo.ipynb">
      <center>
        <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
      </center>
      View source on GitHub
    </a>
  </td>
  <td>
    <a href="https://github.com/Vivswan/AnalogVNN/raw/v1.0.0/docs/_static/AnalogVNN_Demo.ipynb">
      <center>
        <img src="https://www.tensorflow.org/images/download_logo_32px.png" />
      </center>
      Download notebook
    </a>
  </td>
</table>

#### To create 3 layered linear photonic analog neural network with 4-bit [precision](https://analogvnn.readthedocs.io/en/v1.0.0/extra_classes.html#reduceprecision), 0.5 [leakage](https://analogvnn.readthedocs.io/en/v1.0.0/extra_classes.html#leakage-or-error-probability) and [clamp](https://analogvnn.readthedocs.io/en/v1.0.0/extra_classes.html#clamp) normalization:

![3 Layered Linear Photonic Analog Neural Network](analogvnn_model.png)

Python file:
[Sample code](https://github.com/Vivswan/AnalogVNN/blob/v1.0.0/sample_code.py)
and
[Sample code with logs](https://github.com/Vivswan/AnalogVNN/blob/v1.0.0/sample_code_with_logs.py)

## Setting up the Enviroment AnalogVNN

In [ ]:
# Install AnalogVNN with Pip
!pip install analogvnn

In [ ]:
import torch
import analogvnn

import torch.backends.cudnn
import torchvision
from torch import optim, nn
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

from analogvnn.nn.Linear import Linear
from analogvnn.nn.activation.Gaussian import GeLU
from analogvnn.nn.module.FullSequential import FullSequential
from analogvnn.nn.noise.GaussianNoise import GaussianNoise
from analogvnn.nn.normalize.Clamp import Clamp
from analogvnn.nn.precision.ReducePrecision import ReducePrecision
from analogvnn.parameter.PseudoParameter import PseudoParameter
from analogvnn.utils.is_cpu_cuda import is_cpu_cuda

print(f"AnalogVNN version: {analogvnn.__version__}")
print(f"PyTorch version: {torch.__version__}")

is_cpu_cuda.use_cuda_if_available()
torch.manual_seed(0)
device, is_cuda = is_cpu_cuda.is_using_cuda
print(f"Device: {device}")

## Load a dataset

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

In [ ]:
def load_vision_dataset(dataset, path, batch_size, is_cuda=False, grayscale=True):
    dataset_kwargs = {
        'batch_size': batch_size,
        'shuffle': True
    }

    if is_cuda:
        cuda_kwargs = {
            'num_workers': 1,
            'pin_memory': True,
        }
        dataset_kwargs.update(cuda_kwargs)

    if grayscale:
        use_transform = transforms.Compose([
            transforms.Grayscale(),
            transforms.ToTensor(),
        ])
    else:
        use_transform = transforms.Compose([transforms.ToTensor()])

    train_set = dataset(root=path, train=True, download=True, transform=use_transform)
    test_set = dataset(root=path, train=False, download=True, transform=use_transform)
    train_loader = DataLoader(train_set, **dataset_kwargs)
    test_loader = DataLoader(test_set, **dataset_kwargs)

    zeroth_element = next(iter(test_loader))[0]
    input_shape = list(zeroth_element.shape)

    return train_loader, test_loader, input_shape, tuple(train_set.classes)

# Loading Data...
print(f"Loading Data...")
train_loader, test_loader, input_shape, classes = load_vision_dataset(
    dataset=torchvision.datasets.MNIST,
    path="_data/",
    batch_size=128,
    is_cuda=is_cuda
)

## Build a 3 layered linear photonic analog neural network

[`FullSequential`](https://analogvnn.readthedocs.io/en/v1.0.0/autoapi/analogvnn/nn/module/FullSequential/index.html#analogvnn.nn.module.FullSequential.FullSequential) is sequential model where backward graph is the reverse of forward graph.

To add the [Reduce Precision](https://analogvnn.readthedocs.io/en/v1.0.0/extra_classes.html#reduce-precision), [Normalization](https://analogvnn.readthedocs.io/en/v1.0.0/extra_classes.html#normalization), and [Noise](https://analogvnn.readthedocs.io/en/v1.0.0/extra_classes.html#noise) before and after the main Linear layer, `add_layer` function is used.

Leakage definition: [https://analogvnn.readthedocs.io/en/v1.0.0/extra_classes.html#leakage-or-error-probability](https://analogvnn.readthedocs.io/en/v1.0.0/extra_classes.html#leakage-or-error-probability)

In [ ]:
class LinearModel(FullSequential):
    def __init__(self, activation_class, norm_class, precision_class, precision, noise_class, leakage):
        super().__init__()

        self.activation_class = activation_class
        self.norm_class = norm_class
        self.precision_class = precision_class
        self.precision = precision
        self.noise_class = noise_class
        self.leakage = leakage

        self.all_layers = []
        self.all_layers.append(nn.Flatten(start_dim=1))
        self.add_layer(Linear(in_features=28 * 28, out_features=256))
        self.add_layer(Linear(in_features=256, out_features=128))
        self.add_layer(Linear(in_features=128, out_features=10))

        self.add_sequence(*self.all_layers)

    def add_layer(self, layer):
        self.all_layers.append(self.norm_class())
        self.all_layers.append(self.precision_class(precision=self.precision))
        self.all_layers.append(self.noise_class(leakage=self.leakage, precision=self.precision))
        self.all_layers.append(layer)
        self.all_layers.append(self.noise_class(leakage=self.leakage, precision=self.precision))
        self.all_layers.append(self.norm_class())
        self.all_layers.append(self.precision_class(precision=self.precision))
        self.all_layers.append(self.activation_class())
        self.activation_class.initialise_(layer.weight)

Note: [`analogvnn.nn.module.Sequential.Sequential.add_sequence()`](https://analogvnn.readthedocs.io/en/v1.0.0/autoapi/analogvnn/nn/module/Sequential/index.html#analogvnn.nn.module.Sequential.Sequential.add_sequence) is used to create and set forward and backward graphs in AnalogVNN, more information in Inner Workings

In [ ]:
nn_model = LinearModel(
    activation_class=GeLU,
    norm_class=Clamp,
    precision_class=ReducePrecision,
    precision=2 ** 4,
    noise_class=GaussianNoise,
    leakage=0.5
)
print(nn_model)

## Build a WeightModel 

WeightModel is used to parametrize the parameter of LinearModel to simulate photonic weights

[`FullSequential`](https://analogvnn.readthedocs.io/en/v1.0.0/autoapi/analogvnn/nn/module/FullSequential/index.html#analogvnn.nn.module.FullSequential.FullSequential) is sequential model where backward graph is the reverse of forward graph.

In [ ]:
class WeightModel(FullSequential):
    def __init__(self, norm_class, precision_class, precision, noise_class, leakage):
        super().__init__()
        self.all_layers = []

        self.all_layers.append(norm_class())
        self.all_layers.append(precision_class(precision=precision))
        self.all_layers.append(noise_class(leakage=leakage, precision=precision))

        self.eval()
        self.add_sequence(*self.all_layers)

Note: Since the `WeightModel` will only be used for converting the data to analog data to be used in the main `LinearModel`, we can use `eval()` to make sure the `WeightModel` is never been trained

In [ ]:
weight_model = WeightModel(
    norm_class=Clamp,
    precision_class=ReducePrecision,
    precision=2 ** 4,
    noise_class=GaussianNoise,
    leakage=0.5
)
print(weight_model)

Using [`PseudoParameter`](https://analogvnn.readthedocs.io/en/v1.0.0/inner_workings.html#pseudoparameters) to parametrize the parameter

In [ ]:
PseudoParameter.parametrize_module(nn_model, transformation=weight_model)

## Adding accuracy, loss and optimizer

In [ ]:
def cross_entropy_accuracy(output, target) -> float:
    _, preds = torch.max(output.data, 1)
    correct = (preds == target).sum().item()
    return correct / len(output)

In [ ]:
nn_model.loss_function = nn.CrossEntropyLoss()
nn_model.accuracy_function = cross_entropy_accuracy
nn_model.optimizer = optim.Adam(params=nn_model.parameters())

## Compiling the Models

In [ ]:
nn_model.compile(device=device)
weight_model.compile(device=device)
print("Compiled")

## Train and evaluate your model

In [ ]:
for epoch in range(10):
    train_loss, train_accuracy = nn_model.train_on(train_loader, epoch=epoch)
    test_loss, test_accuracy = nn_model.test_on(test_loader, epoch=epoch)

    str_epoch = str(epoch + 1).zfill(1)
    print_str = f'({str_epoch})' \
                f' Training Loss: {train_loss:.4f},' \
                f' Training Accuracy: {100. * train_accuracy:.0f}%,' \
                f' Testing Loss: {test_loss:.4f},' \
                f' Testing Accuracy: {100. * test_accuracy:.0f}%\n'
    print(print_str)

## Conclusion

Congratulations! You have trained a 3 layered linear photonic analog neural network with 4-bit [precision](https://analogvnn.readthedocs.io/en/v1.0.0/extra_classes.html#reduceprecision), 0.5 [leakage](https://analogvnn.readthedocs.io/en/v1.0.0/extra_classes.html#leakage-or-error-probability) and [clamp](https://analogvnn.readthedocs.io/en/v1.0.0/extra_classes.html#clamp) normalization

GitHub: [https://github.com/Vivswan/AnalogVNN](https://github.com/Vivswan/AnalogVNN)

Documentation: [https://analogvnn.readthedocs.io/](https://analogvnn.readthedocs.io/)